In [1]:
%load_ext autoreload
%autoreload 2

# Dependencies

In [2]:
import datasets_handler
import evaluation_metrics
from zeroberto import ZeroBERTo
import gc
import pandas as pd
from datasets import load_dataset, Dataset
from sklearn.preprocessing import LabelEncoder



/Users/alealcoforado/Documents/Projetos/ZeroBERTo/zeroberto/datasets_handler.py:172: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def splitDocuments(docs: pd.Series()) -> list():
0it [00:00, ?it/s]


In [3]:
# df = pd.read_csv('/Users/alealcoforado/Documents/Projetos/Datasets/folhauol/folhauol_clean_df_articles.csv',nrows=4000)
# df.to_csv('/Users/alealcoforado/Documents/Projetos/Datasets/folhauol/folhauol_4000examples_clean_df_articles.csv')

# Data

In [37]:
which_dataset = 'folhauol' 
max_inferences = 100
random_state = 422

le = LabelEncoder()

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)
classes_list = list(raw_data[class_col].unique())
le.fit(raw_data[class_col])
raw_data = evaluation_metrics.Encoder(raw_data,labelEncoder=le,columnsToEncode=[class_col])
data_to_label = raw_data.sample(max_inferences,random_state=random_state).sort_index()


# Parameters

In [38]:
zeroshot_method = "dotproduct"  

zeroshot_config = {
    'similarity_model' : 'sentence-transformers/stsb-xlm-r-multilingual', # model for semantic similarity
    'setfit_model' : 'sentence-transformers/stsb-xlm-r-multilingual',# 'ricardo-filho/bert-base-portuguese-cased-nli-assin-2' # model for contrastive learning and classification
    'dataset':which_dataset, ### which dataset we are using
    'class_col':class_col, ### which column of the dataset has the classes
    'data_col':data_col, ### which column of the dataset has the data
    'split':"zeroshot", ### fewshot (uses real training data) or zeroshot (uses previously selected training data)
    'labeling_method':zeroshot_method, ### how the first labeling should be executed
    'max_inferences':max_inferences, ### sample size from full dataset (number of records)
    'classes':classes_list, ### classes from the dataset
    'template': "Esse texto está relacionado a {}.", ### prompt that will be used for semantic search
    'random_state':random_state, ### random state for reproducibility
    'top_n': 8, ### how many best predictions from zeroshot should be recovered
    'training_examples': 4, ### how many examples per class should be used for training
    "batch_size" : 4, ### contrastive training batch size
    "num_pairs" : 5, ### number of text pairs that will be generated in contrastive learning
    "num_epochs" : 1, ### number of epochs training will be conducted
    # 'keep_body_frozen_setfit': True,     # Unfreeze head and freeze body -> head-only training
    # 'keep_body_frozen_setfit' : False', # Unfreeze head and body -> end-to-end training
    'label_encoder' : le, ### label encoder to use for encoding categorial data into int

    'st_train_epochs': 10, ### sentence-transformer TSDAE fine-tuning --- do not use
    'st_train_batch_size': 20, ### sentence-transformer TSDAE fine-tuning --- do not use
}



# Model

In [15]:
model = ZeroBERTo(classes_list=zeroshot_config['classes'],labeling_dataset=data_to_label,
                embeddingModel=zeroshot_config['similarity_model'],config=zeroshot_config)
       

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


1
queries
3
4


# Labeling

In [39]:
%%time
if (model.config['split']=='zeroshot'): ###  selecting training data
    model.runLabeling()
    model.evaluateLabeling()
    model.getLabelingMetrics()
    model.saveLabelingResults()
    zeroshot_previous_data = datasets_handler.getZeroshotPreviousData( ### load results from labeling step and create training data for contrastive learning
        model.config['dataset'],model.config['class_col'],top_n=model.config['top_n'],exec_time=model.config['exec_time'])

       prediction_code                      prediction
343                  5                   Meio Ambiente
1187                 8      Poder e Política no Brasil
4533                 7      Notícias de fora do Brasil
7146                 0                         Ciência
12548                9  TV, Televisão e Entretenimento
16772                4                         Esporte
19535                3              Equilíbrio e Saúde
25437               10                      Tecnologia
44882                6                         Mercado
55041               11                         Turismo
80192                2                        Educação
top 1: {'accuracy': 0.5454545454545454}
top 2: {'accuracy': 0.35}
top 3: {'accuracy': 0.36}
top 4: {'accuracy': 0.3448275862068966}
top 5: {'accuracy': 0.34375}
top 6: {'accuracy': 0.3235294117647059}
top 7: {'accuracy': 0.3055555555555556}
top 8: {'accuracy': 0.3157894736842105}
top 9: {'accuracy': 0.3}
top 10: {'accuracy': 0.33333333333

/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

predictions_and_probabilities_test_2023_04_09__20_10_14.csv
zeroshot_config_test_2023_04_09__20_10_14.csv
CPU times: user 15.5 s, sys: 1.68 s, total: 17.2 s
Wall time: 34.8 s


Constrative Learning Data

In [41]:
df_train,df_test = datasets_handler.splitDataset(model.labeling_dataset,model.config) 
train_dataset,test_dataset = datasets_handler.buildDatasetDict(df_train,df_test)


/Users/alealcoforado/Documents/Projetos/ZeroBERTo/zeroberto/datasets_handler.py:168: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  train_dataset = Dataset.from_dict(df_train[['text','class_code']].to_dict('list'))


# Training

In [43]:
### overrides config for looping second part of the model
# model.config['num_epochs'] = 5
# model.config['num_pairs'] = 20
# # model.config['keep_body_frozen_setfit'] = False

### build trainer for contrastive learning
model.buildTrainer(train_dataset)

### train
model.contrastive_train()
gc.collect()

### predict labeling dataset with new classifier
# model.getPredictions()

model.predict(test_dataset['text'])
# 5000 sentences --- 4min40s ---sts-xlm-rr
# 5000 sentences ---  ---ricardo-filho/bert-portuguese


/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

{'weighted': [{'accuracy': 0.35}, {'precision': 0.36908794326241146}, {'recall': 0.35}, {'f1': 0.28333559577677225}], 'macro': [{'accuracy': 0.35}, {'precision': 0.24956264775413714}, {'recall': 0.31519157088122607}, {'f1': 0.24773127199597786}]}
metrics_setfit_2023_04_09__20_19_11.csv
config_setfit_2023_04_09__20_19_11.csv
{'similarity_model': 'sentence-transformers/stsb-xlm-r-multilingual', 'setfit_model': 'sentence-transformers/stsb-xlm-r-multilingual', 'dataset': 'folhauol', 'class_col': 'class', 'data_col': 'text', 'split': 'zeroshot', 'labeling_method': 'dotproduct', 'max_inferences': 100, 'classes': ['Poder e Política no Brasil', 'Mercado', 'Notícias de fora do Brasil', 'Esporte', 'Tecnologia', 'Meio Ambiente', 'Equilíbrio e Saúde', 'Educação', 'TV, Televisão e Entretenimento', 'Ciência', 'Turismo', 'Comida'], 'template': 'Esse texto está relacionado a {}.', 'random_state': 422, 'top_n': 8, 'training_examples': 8, 'batch_size': 4, 'num_pairs': 5, 'num_epochs': 1, 'st_train_epoch

In [ ]:
setfit_all_metrics = evaluation_metrics.get_metrics(model.y_pred, test_dataset["class_code"])
print(setfit_all_metrics)

setfit_exec_time  = evaluation_metrics.saveResults(model.config,setfit_all_metrics,None)
print(model.config)